In [1]:
# Import necessary libraries
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import col, concat_ws, collect_list
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import time
import json
from operator import add
import sys

# Add necessary paths
sys.path.append('/home/ubuntu/spark/python/lib/py4j-0.10.9.7-src.zip')
sys.path.append('/home/ubuntu/spark/python')

# Set JDBC driver path
jdbc_driver_path = "/home/ubuntu/postgresql-42.3.5.jar"

# Create a Spark session (reuse or create new)
def create_spark_session(workers, cores):
    try:
        if not SparkSession.builder.getOrCreate().sparkContext._activeContext:
            raise Exception("SparkContext has been stopped.")
    except Exception:
        spark = SparkSession.builder \
            .master("spark://192.168.2.114:7077") \
            .appName("Lyrics Sentiment Analysis") \
            .config("spark.executor.cores", cores) \
            .config("spark.executor.instances", workers) \
            .config("spark.jars", jdbc_driver_path) \
            .config("spark.driver.extraClassPath", jdbc_driver_path) \
            .config("spark.executor.extraClassPath", jdbc_driver_path) \
            .config("spark.sql.shuffle.partitions", 200) \
            .config("spark.executor.memory", "4g") \
            .config("spark.driver.memory", "4g") \
            .config("spark.ui.port", "4044") \
            .getOrCreate()

    sc = spark.sparkContext.setLogLevel("ERROR")
    return spark

# Sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Database connection info
DB_URL = "jdbc:postgresql://192.168.2.114:5432/lyricsdb"
DB_PROPERTIES = {"user": "sparkuser", "password": "9567", "driver": "org.postgresql.Driver"}

def analyze_sentiment(text):
    score = analyzer.polarity_scores(text)["compound"]
    return "positive" if score > 0.05 else "negative" if score < -0.05 else "neutral"

def run_experiment(workers, data_fraction, cores):
    spark = create_spark_session(workers, cores)

    print(f"Running with workers={workers}, data_fraction={data_fraction}, cores={cores}...")

    sqlContext = SQLContext(spark)
    words_df = sqlContext.read.jdbc(DB_URL, "words", properties=DB_PROPERTIES)

    total_sentiment_counts = {"positive": 0, "negative": 0, "neutral": 0}

    start_time = time.time()

    for partition in [('a', 'g'), ('h', 'n'), ('o', 'u'), ('v', 'z')]:
        print(f"Loading partition: {partition}")
        table = "lyrics_" + partition[0] + "_" + partition[1]
        lyrics_df = sqlContext.read.jdbc(DB_URL, table, properties=DB_PROPERTIES)

        print("Reconstructing lyrics...")
        full_lyrics_df = lyrics_df.join(words_df, "word_id") \
            .groupBy("track_id") \
            .agg(concat_ws(" ", collect_list("word")).alias("lyrics"))
        print("Lyrics reconstructed.")

        print(f"full_lyrics_df: {full_lyrics_df.count()}")

        full_lyrics_df = full_lyrics_df.repartition(100)
        sample_df = full_lyrics_df.sample(fraction=data_fraction, seed=42)

        print(f"Sample data rows after fractioning: {sample_df.count()}")

        print("Data loaded successfully.")

        sentiment_udf = spark.udf.register("sentiment", analyze_sentiment)
        sentiment_df = sample_df.withColumn("sentiment", sentiment_udf(col("lyrics")))
        print(f"Total rows: {sentiment_df.count()}")

        sentiment_df = sentiment_df.repartition(200)

        sentiment_counts = sentiment_df.groupBy("sentiment").count().collect()

        print(sentiment_counts, "for partition", partition)

        # Update total sentiment counts
        for row in sentiment_counts:
            sentiment = row["sentiment"]
            count = row["count"]
            total_sentiment_counts[sentiment] += count

    elapsed_time = time.time() - start_time

    print("\nFinal aggregated sentiment counts across all partitions:")
    for sentiment, count in total_sentiment_counts.items():
        print(f"{sentiment}: {count}")

    spark.stop()

    return {
        "workers": workers,
        "data_fraction": data_fraction,
        "cores": cores,
        "time": elapsed_time,
        "sentiment_counts": total_sentiment_counts
    }
if __name__ == "__main__":
    # List of parameters to loop through
    workers_list = [1, 2, 4]
    fractions = [0.01, 0.04, 0.08]
    cores_list = [1, 2, 4]

    results = []

    for workers in workers_list:
        for data_fraction in fractions:
            for cores in cores_list:
                result = run_experiment(workers, data_fraction, cores)
                if result is None:  # If process was terminated due to empty DataFrames
                    continue
                results.append(result)

    # Write results to JSON file
    with open("experiment_results.json", "w") as outfile:
        json.dump(results, outfile, indent=4)

    print("Experiment results saved to 'experiment_results.json'.")

    # result = run_experiment(workers = 4, data_fraction = 0.01, cores = 1)
    # print(result)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/25 21:02:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/25 21:02:45 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
/home/ubuntu/.local/lib/python3.10/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Running with workers=1, data_fraction=0.01, cores=1...
Loading partition: ('a', 'g')
Reconstructing lyrics...
Lyrics reconstructed.


full_lyrics_df: 56960


Sample data rows after fractioning: 567
Data loaded successfully.


Total rows: 567


...

[Row(sentiment='positive', count=2593), Row(sentiment='neutral', count=239), Row(sentiment='negative', count=1767)] for partition ('o', 'u')
Loading partition: ('v', 'z')
Reconstructing lyrics...
Lyrics reconstructed.


full_lyrics_df: 32082


Sample data rows after fractioning: 2603
Data loaded successfully.


Total rows: 2603


[Row(sentiment='positive', count=1499), Row(sentiment='negative', count=980), Row(sentiment='neutral', count=126)] for partition ('v', 'z')

Final aggregated sentiment counts across all partitions:
positive: 9308
negative: 6224
neutral: 883
Experiment results saved to 'experiment_results.json'.
